# Embedding the Documents

In [1]:
import os
import json
import numpy as np
import threading
import time
from cltk import NLP


c:\environments\envs\alchem\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class NumpyArrayEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [3]:
embedding_dir = "./data/embedding/"
doc_dir = "./data/"

In [4]:
def nlp_file(filename, fpath, tpath):
    print(f"--- Start Process for {filename}, {time.ctime()}")
    with open(fpath) as jf:
        wdoc = json.load(jf)
    cltk_nlp = NLP(language='lat', suppress_banner=True)

    for bk in wdoc:
        cltk_doc = cltk_nlp.analyze(bk["texts"])
        bk["sentences"] = cltk_doc.sentences_strings
        bk["sentence_embeddings"] = cltk_doc.sentence_embeddings
        print(f"------ Finished {filename}, {bk['title']}, {bk['bks']}")
    with open(tpath, 'w') as jf:
        json.dump(wdoc, jf, cls=NumpyArrayEncoder)
    print(f"--- End Process for {filename}, {time.ctime()}")


In [5]:
from concurrent.futures import ThreadPoolExecutor


In [6]:
with ThreadPoolExecutor(max_workers=18) as pool:
    def get_result(future):
        print(future.result())

    for tname in os.listdir(doc_dir):
        fpath = os.path.join(doc_dir, tname)
        fname, ext = os.path.splitext(tname)

        if not os.path.isfile(fpath) or ext != '.json':
            #print(f"{fname} is not a json file")
            continue
        tpath = os.path.join(embedding_dir, fname+'_embedding'+ext)
        pool.submit(nlp_file, fname, fpath, tpath).add_done_callback(get_result)
        

    


FileNotFoundError: [WinError 3] 系统找不到指定的路径。: './data/'